In [2]:
import forallpeople
forallpeople.environment('Structural',top_level=True)
#Useful extra units
cm = 1e-2*m #type: ignore
from IPython.display import Markdown, display
import handcalcs.render

Calculations
Let the force be represented as:
$$F=ma$$
Where:
- $m$ is the mass in $\text{kg}$
- $a$ is the acceleration in $\text{m/s}^2$

In [5]:
# Example calculation
mass=100*kg
a = 9.81*m/s**2
force = mass * a
force=5*kN
display(Markdown(f"The result is: $F$={round(force,1)}"))
t = 3.75*mm #Espesor
H = 300*mm #Altura
B = 100*mm #Ancho
I_x = B*H**3/12 - (B - t)*(H - t)**3/12
I_y = H*B**3/12 - (H - t)*(B - t)**3/12
display(Markdown(f"The result is: $I_x$={round(I_x,0)}"))
display(Markdown(f"The result is: $I_y$={round(I_y,0)}"))
# Cambio de unidad
force.to(unit_name="kip")


The result is: $F$=5.0 kN

The result is: $I_x$=16457503 mm⁴

The result is: $I_y$=2986995 mm⁴

1.124 kip

In [16]:
%%capture
!jupyter nbconvert --to markdown --no-input report_example.ipynb

In [33]:
%%capture
!jupyter nbconvert --no-input report_example.ipynb --to html

In [10]:
%%capture
!jupyter nbconvert --to pdf --no-input report_example.ipynb

In [11]:
%%capture
!jupyter nbconvert --to pdf report_example.ipynb

In [6]:
%%render 2
mass=100*kg
a = 9.81*m/s**2
force = mass * a

<IPython.core.display.Latex object>

In [7]:
%%render 2
mass=100*kg
a = (9.81*m/s**2)
force = (mass * a)

<IPython.core.display.Latex object>

In [8]:
%%render params 2 
mass=100*kg
a = 9.81*m/s**2
force = mass * a

<IPython.core.display.Latex object>

In [9]:
%%render symbolic
mass=100*kg #Comment in line
a = 9.81*m/s**2
force = mass * a


<IPython.core.display.Latex object>

In [10]:
%%render params 2
force = mass * a

<IPython.core.display.Latex object>

In [11]:
%%render params 2
#Tubo 200x100x3.75mm
t = 3.75*mm #Espesor
H = 200*mm #Altura
B = 100*mm #Ancho

<IPython.core.display.Latex object>

In [12]:
%%render
I_x = B*H**3/12 - (B - t)*(H - t)**3/12
I_y = H*B**3/12 - (H - t)*(B - t)**3/12

<IPython.core.display.Latex object>

In [39]:
import sys
import os
# Añadir el directorio raíz al PYTHONPATH
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../structurelab')))

from structurelab.settings import Settings
from structurelab.material import create_concrete, SteelBar
from structurelab.section import Beam

from pylatex import Command, Document, Math, Matrix, Section, Subsection, VectorName, NoEscape, Figure, LongTable
from pylatex.utils import NoEscape
import matplotlib
matplotlib.use("Agg")  # Not to use X server. For TravisCI.
import matplotlib.pyplot as plt  # noq



class MyDocument(Document):
    def __init__(self, documentclass='article', document_options=['a4paper','spanish','10pt'], geometry_options={"left": "20mm","right": "20mm"}):
        # Pasar las opciones al constructor de la clase base Document
        super().__init__(documentclass=documentclass, document_options=document_options, geometry_options=geometry_options)
        self.preamble.append(Command("usepackage","mathtools"))
        self.preamble.append(Command("title", "Viga rectangular a flexión - ACI 318-19"))
        self.preamble.append(Command("author", "StructureLab"))
        self.preamble.append(Command("date", NoEscape(r"\today")))
        self.append(NoEscape(r"\maketitle"))


def main():
    # Ejemplo de uso
    concrete=create_concrete(name="H30",f_c=30*MPa, design_code="ACI 318-19") # type: ignore
    steelBar=SteelBar(name="ADN 420", f_y=420*MPa) # type: ignore
    section = Beam(
        name="V-40x50",
        concrete=concrete,
        steelBar=steelBar,
        width=400 * mm,  # type: ignore
        depth=500 * mm,  # type: ignore
    )

    print(f"Nombre de la sección: {section.get_name()}")

if __name__ == "__main__":
    main()

TypeError: SteelBar.__init__() got an unexpected keyword argument 'f_y'